In [1]:
#imports
from configFile import *
from configFile import PartitioningType
from utils import compareBM25List
from utils import splitQueryAndTime
from utils import Tokenizer
import utils
from node import Node
import math

docLengthsDictionary = utils.docLengthsDictionary
docDic = utils.docDic

### CONFIGS
partitioningType = PartitioningType.TERMBASED
partitionNum = 32

wordListFileName = '../webData/wordlist.txt'
queryListFileName = '../webData/10000.topics'
### 

### GLOBALS
cumulativeDic = {}
documentDict = {}
###
#Reads the binary file and prints the entries

In [2]:

def docLengths():
    global docLengthsDictionary
    with open('../webData/doc_lengths.txt') as f:
        lines = f.readlines()
        for line in lines:
            line = line.split(' ')
            docLengthsDictionary[line[0]] = line[1]
            

def readBinaryFile():
    global docDic
    k = 0
    with open('../webData/entry.bin', 'rb') as f:
        # data = f.seek(8*seekLength)
        for (key,(startPos,pll)) in cumulativeDic.items():
            data = (f.read(pll*8).hex(':', 4)).split(':')
            # k+=1
            # if(k==1000):
            #     break
            j = 0
            docId = -1
            docDic[key]= {}
            for i in data:
                newData = bytearray.fromhex(i)[::-1].hex(':',4)
                if(j%2 == 0):
                    docId = int(newData, 16)
                else:
                    docDic[key][docId] = int(newData, 16)
                j+=1

    # return docDic



def wordListReader(filename):
    wordList = []

    with open(filename) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=' ')
        for row in csv_reader:
            wordList.append(row)
    
    return wordList


def queryListReader(filename):
    queryList = []

    with open(filename, encoding='latin1') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=':')
        for row in csv_reader:
            querylistRow = row[1].translate(str.maketrans('','',string.punctuation))
            queryList.append(querylistRow.split(' '))
    
    return queryList


def termBasedPartitioner(wordList, partitionNum): 
    partitionDictList = []
    for i in range(partitionNum):
        newDict = dict()
        partitionDictList.append(newDict)
    termMap = {}
    
    cumulativeArr = [0]
    global cumulativeDic
    cumulativeDic = {}

    listIndex = 0
    while listIndex < len(wordList):
        partitionIndex = listIndex % partitionNum
        partitionDictList[partitionIndex][wordList[listIndex][0]] = wordList[listIndex][1]
        termMap[wordList[listIndex][0]] = partitionIndex

        cumulativeDic[wordList[listIndex][0]] = (cumulativeArr[-1], int(wordList[listIndex][1]))
        cumulativeArr.append(cumulativeArr[-1]+int(wordList[listIndex][1]))

        listIndex = listIndex + 1

    return partitionDictList, termMap, cumulativeDic


def pseudoPartition(partitionDict, query):
    totalCost = 0
    minCost = sys.maxsize

    for term in query:
        PLL = partitionDict.get(term)
        if PLL is not None:
            PLL = float(PLL)
            if PLL > 0:
                totalCost = totalCost + PLL
                minCost = min(minCost, PLL)
    
    if minCost == sys.maxsize:
        minCost = 0

    return totalCost, minCost


def broker(query, nodeList, termMap):
    totalCost = 0
    partitionCostList = []
    totalPartitionCost = 0
    minPartitionCost = sys.maxsize

    nodeTermList = [[] for _ in range(len(nodeList))]

    for term in query:
        # NOTE: Check starting termMap Value index. 0 or 1
        if term in termMap.keys():
            nodeTermList[termMap[term]].append(term) 


    nodeTopKDocList = []
    partitionIndex = 0
    for node in nodeList:
        if len(nodeTermList[partitionIndex]) != 0:
            nodeTopKDoc = node.calculateTopKDoc(nodeTermList[partitionIndex])
            nodeTopKDocList.append(nodeTopKDoc)
            # print('nodeTopKDoc', nodeTopKDoc)

        totalCost = totalCost + minPartitionCost
        partitionCostList.append(totalPartitionCost)
        partitionIndex = partitionIndex + 1
    
    topKDoc = compareBM25List(nodeTopKDocList)
    
    return topKDoc, totalCost, partitionCostList

In [13]:
wordList = wordListReader(wordListFileName)
queryList, timeStamps = splitQueryAndTime()
trainLength = math.floor(len(queryList)*0.70)
queryListTest = queryList[trainLength:]
queryListTrain = queryList[:trainLength]

timeStampsTest = timeStamps[trainLength:]
timeStampsTrain = timeStamps[:trainLength]

nodeList = []
global cumulativeDic
if partitioningType.value == PartitioningType.TERMBASED.value:
    partitioner = termBasedPartitioner

[partitionDictList, termMap, cumulativeDic] = partitioner(wordList, partitionNum)
docLengths()
readBinaryFile()


In [61]:
# TODO: CACHE SIZE NUMBERS WILL BE HERE
initialCacheSize = 40000 #In terms of PL length #5000 = 40kb
maxCachePLLength = 400
nodeList = []
for nodeNum in range(partitionNum):
    currentNode = Node(partitionDictList[nodeNum])
    currentNode.EmptyCacheSize = initialCacheSize
    nodeList.append(currentNode)

queryIndex = 0
brokerCost = 0
partitioncCostList = len(partitionDictList) * [0] ####



### Caching Implementation

In [62]:
import cacheFunctions
import importlib
importlib.reload(cacheFunctions)

### NON-JUPYTER CODE
# nodeFrequencyList = cacheFunctions.nodeFrequencyList 
# meansForIntervals = cacheFunctions.meansForIntervals
# frequencyDict = cacheFunctions.frequencyDict

# TEMP CLEARS
cacheFunctions.nodeFrequencyList.clear()
cacheFunctions.frequencyDict.clear()
cacheFunctions.nodeFrequencyList = [[] for i in range(32)]

termCaches = [{} for i in range(32)]

cacheFunctions.frequencyDict = (cacheFunctions.queryFrequencies(queryListTrain))
period = 16
interval = 8
# cacheFunctions.queryFrequencyStability(period, interval, queryListTrain, timeStampsTrain)

cacheFunctions.everyNodeFrequency(termMap) # Frequency
# cacheFunctions.stabilityFunction(termMap) # Stability

# print(cacheFunctions.nodeFrequencyList)

def cacheTermBM25():
    lengthNFList = len(cacheFunctions.nodeFrequencyList)
    for i in range(lengthNFList):
        idx = 0
        for term in cacheFunctions.nodeFrequencyList[i]:
            cacheFunctions.nodeFrequencyList[i][idx][1][1] =  utils.BM25Algorithm(term[0])
            cacheFunctions.nodeFrequencyList[i][idx][1][1].sort(key=lambda x: x[1], reverse=True)
            idx+=1


cacheTermBM25()



In [63]:
for node,idx in zip(nodeList,range(len(nodeList))):
    for cache in cacheFunctions.nodeFrequencyList[idx]:
        length = len(cache[1][1])
        if(length > maxCachePLLength):
            length = maxCachePLLength

        if(node.EmptyCacheSize >= length):        
            node.Cache[cache[0]] = [cache[1][0],cache[1][1][:length]]
            node.EmptyCacheSize -= length
        elif(node.EmptyCacheSize > 0):
            node.Cache[cache[0]] =  [cache[1][0],cache[1][1][:node.EmptyCacheSize]]
            node.EmptyCacheSize = 0
        else:
            break
    # print(len(node.Cache['jobs'][1]))

### MAIN Function 

In [64]:
for query in queryListTest:
    
   #  print('before tokenizer',query)
    newQuery = []
    newQuery = Tokenizer.tokenize(query)
   #  print('after tokenizer= =', newQuery)
    [topKDoc, currBrokerCost, currPartitionCostList] = broker(newQuery, nodeList, termMap)
    brokerCost = brokerCost + currBrokerCost
    partitioncCostList = list(map(add, partitioncCostList, currPartitionCostList))
    queryIndex = queryIndex + 1
   #  print('topKDoc',topKDoc[:200])
    
   #  sys.exit(0)
for node,idx in zip(nodeList,range(len(nodeList))):
   print('Node id => ',idx, ', percentage hit =>', node.cacheHitCounter/node.totalRecievedTerm , ', cache hit counter=> ', node.cacheHitCounter, ', total Recieved Term => ' , node.totalRecievedTerm)    

Node id =>  0 , percentage hit => 0.30349099099099097 , cache hit counter=>  539 , total Recieved Term =>  1776
Node id =>  1 , percentage hit => 0.27049754730203224 , cache hit counter=>  386 , total Recieved Term =>  1427
Node id =>  2 , percentage hit => 0.37141249296567247 , cache hit counter=>  660 , total Recieved Term =>  1777
Node id =>  3 , percentage hit => 0.30297029702970296 , cache hit counter=>  612 , total Recieved Term =>  2020
Node id =>  4 , percentage hit => 0.5 , cache hit counter=>  1197 , total Recieved Term =>  2394
Node id =>  5 , percentage hit => 0.26517083120856705 , cache hit counter=>  520 , total Recieved Term =>  1961
Node id =>  6 , percentage hit => 0.3789858860428646 , cache hit counter=>  725 , total Recieved Term =>  1913
Node id =>  7 , percentage hit => 0.27781013395457194 , cache hit counter=>  477 , total Recieved Term =>  1717
Node id =>  8 , percentage hit => 0.26554621848739496 , cache hit counter=>  474 , total Recieved Term =>  1785
Node id 